In [1]:
from pydub import AudioSegment

def load_audio(file_path):
    # Load audio file
    audio = AudioSegment.from_file(file_path)
    # Convert to mono and set sample rate to 16kHz (if needed)
    audio = audio.set_channels(1).set_frame_rate(16000)
    # Export as WAV
    audio.export("temp_audio.wav", format="wav")
    return "temp_audio.wav"

audio_file = load_audio("REC30AirportTravel.wav")  # Replace with your file path

In [3]:
import os

# Set FFmpeg path manually
os.environ["PATH"] += os.pathsep + "C:\\ffmpeg\\bin"  # Adjust this path if needed

# Now, run your Spleeter script
from spleeter.separator import Separator

def separate_voices(audio_file):
    try:
        separator = Separator('spleeter:2stems')
        separator.separate_to_file(audio_file, "output_directory")
        print("Voices separated and saved in 'output_directory'.")
    except Exception as e:
        print(f"An error occurred: {e}")

audio_file = 'REC30AirportTravel.wav'
separate_voices(audio_file)


INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models\\2stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Could not find trained model in model_dir: pretrained_models\2stems, running initialization to predict.
Instructions for updating:
Use output_s

In [13]:
# from spleeter.separator import Separator

# def separate_voices(audio_file):
#     try:
#         # Initialize Spleeter separator (2 stems: vocals and accompaniment)
#         separator = Separator('spleeter:2stems')
        
#         # Separate voices
#         separator.separate_to_file(audio_file, "output_directory")
        
#         print("Voices separated and saved in 'output_directory'.")
#     except Exception as e:
#         print(f"An error occurred: {e}")

# # Replace 'your_audio_file.mp3' with the path to your audio file
# audio_file = 'REC30AirportTravel.wav'
# separate_voices(audio_file)

An error occurred: ffmpeg binary not found


In [4]:
import torch
import torchaudio

def transcribe_audio(audio_file):
    # Load pre-trained Wav2Vec 2.0 model
    bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
    model = bundle.get_model()
    waveform, sample_rate = torchaudio.load(audio_file)

    # Resample if necessary
    if sample_rate != bundle.sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=bundle.sample_rate)
        waveform = resampler(waveform)

    # Transcribe
    with torch.inference_mode():
        emissions, _ = model(waveform)
        tokens = torch.argmax(emissions, dim=-1)
        transcription = bundle.decode(tokens[0])

    return transcription

In [6]:
import os
import torch
import torchaudio
from pydub import AudioSegment
from spleeter.separator import Separator

# Ensure FFmpeg is accessible
os.environ["PATH"] += os.pathsep + "C:\\ffmpeg\\bin"  # Adjust if needed

def load_audio(file_path):
    """Load and preprocess audio: convert to mono and 16kHz WAV."""
    try:
        audio = AudioSegment.from_file(file_path)
        audio = audio.set_channels(1).set_frame_rate(16000)
        processed_audio = "temp_audio.wav"
        audio.export(processed_audio, format="wav")
        return processed_audio
    except Exception as e:
        print(f"Error processing audio: {e}")
        return None

def separate_voices(audio_file):
    """Separate vocals and accompaniment using Spleeter."""
    try:
        separator = Separator('spleeter:2stems')
        output_dir = "output_directory"
        os.makedirs(output_dir, exist_ok=True)
        separator.separate_to_file(audio_file, output_dir)
        print(f"Voices separated and saved in '{output_dir}'.")
        return os.path.join(output_dir, "vocals.wav"), os.path.join(output_dir, "accompaniment.wav")
    except Exception as e:
        print(f"Error in Spleeter: {e}")
        return None, None

def transcribe_audio(audio_file):
    """Transcribe audio using Wav2Vec 2.0."""
    try:
        bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
        model = bundle.get_model()
        waveform, sample_rate = torchaudio.load(audio_file)

        # Resample if necessary
        if sample_rate != bundle.sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=bundle.sample_rate)
            waveform = resampler(waveform)

        # Transcription
        with torch.inference_mode():
            emissions, _ = model(waveform)
            tokens = torch.argmax(emissions, dim=-1)
            transcription = bundle.decode(tokens[0])

        return transcription
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

# Processing pipeline
input_audio = "segment_1.wav"  # Replace with actual file path
processed_audio = load_audio(input_audio)

if processed_audio:
    vocals, accompaniment = separate_voices(processed_audio)
    if vocals and accompaniment:
        speaker1_text = transcribe_audio(vocals)
        speaker2_text = transcribe_audio(accompaniment)

        print("\nTranscription Results:")
        print(f"Speaker 1 (Vocals): {speaker1_text}")
        print(f"Speaker 2 (Accompaniment): {speaker2_text}")


INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models\\2stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Could not find trained model in model_dir: pretrained_models\2stems, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:

In [7]:
def format_conversation(speaker1_text, speaker2_text):
    conversation = []
    # Split sentences (assuming sentences are separated by periods)
    speaker1_sentences = speaker1_text.split('. ')
    speaker2_sentences = speaker2_text.split('. ')
    # Format conversation
    for i in range(max(len(speaker1_sentences), len(speaker2_sentences))):
        if i < len(speaker1_sentences):
            conversation.append(f"Speaker 1: {speaker1_sentences[i].strip()}")
        if i < len(speaker2_sentences):
            conversation.append(f"Speaker 2: {speaker2_sentences[i].strip()}")
    return "\n".join(conversation)

formatted_conversation = format_conversation(speaker1_text, speaker2_text)
print(formatted_conversation)

AttributeError: 'NoneType' object has no attribute 'split'